# Retinal Disease Detection and Classification

## By: Jalen Wu, Yechan Na, Jonathan Zhang

__Project Description:__

The goal of this project is to develop a machine learning model capable of detecting retinal diseases by analyzing fundus images of the eye. Using computer vision and deep learning techniques, the model assists in early detection and diagnosis of retinal disease(s).

__Applications and Impact:__

This project could be used for clinical screening to help ophthalmologists identify diseases and improve efficiency on identifying these diseases. This automated detection system has the potential to make medical imaging diagnostics more accessible.

__What we hope to achieve:__

We hope to build a model that takes in images of the eye and accurately predicts whether an individual’s eyes are healthy or showing signs of disease. To quantify the effectiveness of our model, we will be measuring metrics such as F1 score, precision, recall, loss, and accuracy of our models and graphing them as a function of how many epochs we run on our training data (all of these scores should increase in subsequent epochs).

- __Dataset__: https://www.kaggle.com/datasets/andrewmvd/retinal-disease-classification/data
- __References__: 
    - https://www.mdpi.com/2306-5729/6/2/14
    - https://jamanetwork.com/journals/jama/fullarticle/2588763
    

In [ ]:
# imports

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
import os
from torch.utils.data import Dataset
import pandas as pd
from skimage import io, transform
from torch import tensor
from PIL import Image
from torch import flatten
from nltk.metrics.scores import (precision, recall, f_measure, accuracy)
from tqdm import tqdm

Importing Retinal Disease dataset from Kaggle

In [41]:
# Import Dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/retinal-disease-classification")

print("Path to dataset files:", path)

Path to dataset files: /Users/jalenwu/.cache/kagglehub/datasets/andrewmvd/retinal-disease-classification/versions/1


Setting the device to run our model

In [42]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


Defining a Transform function to be applied to all images in our dataset and verifying success of dataset download.

In [ ]:
# X = 1424 x 2144 x 3 : h x w x colors
# y = label
transform = transforms.Compose([
    transforms.Resize((int(1424 / 8), int(2144 / 8))), # Standardize image dimensions
    transforms.ToTensor(),          # Convert images to PyTorch tensors
])

base_directory = path  
print(base_directory)

if os.path.exists(base_directory):
    print("Directory exists and its contents are:")
    print(os.listdir(base_directory))
else:
    print("Directory does not exist.")

/Users/jalenwu/.cache/kagglehub/datasets/andrewmvd/retinal-disease-classification/versions/1
Directory exists and its contents are:
['.DS_Store', 'Evaluation_Set', 'Training_Set', 'Test_Set']


Defining constants for a single point of control

In [44]:
# Constants (paths that will be accessed later in the project)
BASE_DIRECTORY = path
TRAINING_DIRECTORY = os.path.join(BASE_DIRECTORY, 'Training_Set', 'Training_Set', 'Training')
TRAINING_LABELS = os.path.join(BASE_DIRECTORY, 'Training_Set', 'Training_Set', 'RFMiD_Training_Labels.csv')
TESTING_DIRECTORY = os.path.join(BASE_DIRECTORY, 'Test_Set', 'Test_Set', 'Test')
TESTING_LABELS = os.path.join(BASE_DIRECTORY, 'Test_Set', 'Test_Set', 'RFMiD_Testing_Labels.csv')

Initialized training and testing labels for images in Pandas DataFrames to add support for data manipulation

In [45]:
# Delegated the dataframe creation outside of the dataset class
training_labels_df = pd.read_csv(TRAINING_LABELS)
testing_labels_df = pd.read_csv(TESTING_LABELS)

In [48]:
training_labels_df.head()

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
training_labels_df = training_labels_df.filter(['Disease_Risk'])
testing_labels_df = testing_labels_df.filter(['Disease_Risk'])

,Disease_Risk
0,1
1,1
2,1
3,1
4,1


Created a custom Dataset class that extends the abstract Dataset class provided by PyTorch. 
- This step was necessary as the built-in Dataset classes offered by PyTorch were not compatible with the local file structure of our dataset.

In [50]:
class MultiClassDataset(Dataset):
    """
    Supports structure of given dataset (images in child folder and labels in csv format).
    """
    def __init__(self, label_csv_file, image_directory, transform=None):
        """
        Args:
            label_csv_file (DataFrame): Dataframe of the CSV label file.
            image_directory (str): Directory with eye images.
            transform (callable, optional): transform function to be applied to each image.
        """
        self.label_csv_file = label_csv_file
        self.image_directory = image_directory
        self.transform = transform
    
    def __len__(self):
        return len(self.label_csv_file)    

    def __getitem__(self, index):
        """
        Args:
            index: the index of the image/label pair we want to retrieve
            
        Returns:
            image_and_label (dict): A dictionary containing the image and its corresponding label at the requested index.
        """

        # The images are PNG and one-indexed (1.png, 2.png, 3.png, ...)
        image_path = os.path.join(self.image_directory, str(index + 1) +'.png')
        
        image_label = self.label_csv_file.loc[index]
        image_label = image_label.to_numpy()
        image_label = tensor(image_label)
        
        # Loads image at the given path.
        current_image = io.imread(image_path)
        
        if current_image is None or current_image.size == 0:
            raise ValueError(f"Failed to load image: {image_path}")
       
        # Convert to PIL Image
        current_image = Image.fromarray(current_image)
        
        # If transform function passed in, apply transform to image
        if self.transform:
            transformed_image = self.transform(current_image)
            return {'image': transformed_image, 'label': image_label}
        
        return {'image': current_image,'label': image_label}

Initialized Training dataset and loader for later use in a neural network

In [51]:
# Initializing dataset and loader.
train_dataset = MultiClassDataset(label_csv_file=training_labels_df, image_directory=TRAINING_DIRECTORY, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64)

# Visualize loader
res = next(iter(train_loader))


Verifying correct initialization of items contained in data loader

In [52]:
print(res['image'].shape)
print(res['label'].shape)

print(res['label'][0])
print(res['label'][1])
print(res['label'][2])
print(res['label'][3])
print(res['image'][0][0])

torch.Size([64, 3, 178, 268])
torch.Size([64, 1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [53]:
# image tensor format: [batch_size, channels, height, width]
print(res['image'][0].shape)

torch.Size([3, 178, 268])


Construction of our Convolutional Neural Network
- 3 groupings of convolutional layer, ReLU activation layer, and pooling layer.
- Flattening to transfer data to fully connected layers.
- 4 fully connected layers, each followed by a ReLU activation layer.
- 1 output layer, utilizing sigmoid activation to classify the output of the network.

The forward function is called every time a set of data is passed through the network.

In [54]:
# Original Network
class Conv_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(8640, 256)
        self.fc2 = nn.Linear(256, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 23)
        self.fc5 = nn.Linear(23, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x

conv_model = Conv_NN()

Definition of our training function

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    """
    Trains the model for one epoch.
    
    Args:
        dataloader (DataLoader): DataLoader object with training data
        model (nn.Module): Model to be trained
        loss_fn (nn.Module): Loss function to be used
        optimizer (torch.optim): Optimizer to be used
    
    Returns:
        float: Average loss for the current epoch
    """
    model.train()
    size = len(dataloader.dataset)
    total_loss = 0.0
    
    for i, data in tqdm(enumerate(dataloader)):
        # get the inputs; data is a dict of [inputs, labels]
        inputs = data['image']
        labels = data['label']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()

        # print statistics
        total_loss += loss.item()
    
    return total_loss/size

Creation of test dataset and loader for testing data

In [56]:
test_dataset = MultiClassDataset(label_csv_file=testing_labels_df, image_directory=TESTING_DIRECTORY, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64)

Helper function to determine precision, recall, f1, and accuracy metrics when given a list of predictions and a list of true values (common measures of model effectiveness)

In [ ]:
def PRFA(predictions, answers):
    """
    Returns precision, recall, f1, and accuracy of the predictions.
    
    Args:
        predictions (list): List of predicted labels
        answers (list): List of actual labels
    """
    pred_indices = [x for x in range(len(predictions)) if predictions[x] == 1]
    actual_indices = [y for y in range(len(answers)) if answers[y] == 1]

    temp_precision = precision(set(pred_indices), set(actual_indices)) # actual labels vs. predicted labels
    temp_recall = recall(set(pred_indices), set(actual_indices))
    temp_f1 = f_measure(set(pred_indices), set(actual_indices))
    temp_accuracy = accuracy(answers, predictions)
    return (temp_precision, temp_recall, temp_f1, temp_accuracy)


In [ ]:
accuracy_values = []

def get_preds(dataloader, model):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader)):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['image']
            l = data['label']
            
            labels.append(l)
            preds.append(model(inputs))
    return torch.cat(preds), torch.cat(labels)


def get_clamped_preds(preds, t):
    """
    Takes a dataloader for the test data and model
    Returns the Precision, Recall, F1 Score, and Accuracy of the model as a tuple
    """
    
    new_preds = torch.tensor(np.zeros([len(preds)], dtype=int))
    
    # Goes through all images in preds
    for img_in_batch in range(len(preds)): 
        # Goes through all disease labels in the image
        # Clamp the values to 0 or 1
        if preds[img_in_batch] > t:
            new_preds[img_in_batch] = 1
        else:
            new_preds[img_in_batch] = 0
    return new_preds


def test(new_preds, labels):
    size = len(new_preds)
    correct = 0
    for i in range(len(new_preds)):
        if new_preds[i] == labels[i]:
            correct += 1

    accuracy = correct / size
    print("Accuracy: ", accuracy)
    return PRFA(new_preds, labels)

Optimizer: Stochastic Gradient Descent with Momentum
- learning rate set to 0.001 
- momentum (beta) set to 0.9

$$
w_{t+1} = w_t - V_t
$$

$$
V_t = V_{t-1} + (1 - \beta) \frac{\partial L}{\partial w_t} 
$$


In [ ]:
import torch.optim as optim

loss_fn = nn.BCELoss()

# Changed learning rate from 0.001 to 0.01
optimizer = optim.SGD(conv_model.parameters(), lr=0.001, momentum=0.9)

Training the Neural Network
- Maximum number of epochs: 40
- Each epoch runs train, then test
- PRFA scores are saved and written to .txt file for later use or reference

In [61]:
epochs = 40
conv_model_loss = []
p_scores = []
r_scores = []
f_scores = []
a_scores = []

#conv_model = Conv_NN()
#conv_model.load_state_dict(torch.load("5th_conv_model.pth", weights_only=True))

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    conv_loss = train(train_loader, conv_model, loss_fn, optimizer)
    print("Convolutional Model Loss: ", conv_loss)
    
    # Early stop if loss has increased.
    if t > 3 and conv_loss > conv_model_loss[-1]:
        break
        
    preds, labels = get_preds(test_loader, conv_model)
    avg = (max(preds) + min(preds)) / 2
    # print(avg)
    new_preds = get_clamped_preds(torch.flatten(preds), avg)
    # print(new_preds[0])
    p, r, f, a = test(new_preds, labels)
    
    print("prfa: ")
    print(p, r, f, a)
    p_scores.append(p)
    r_scores.append(r)
    f_scores.append(f)
    a_scores.append(a)
    
     # if the accuracy has decreased, early stop
    # if len(a_scores) > 1 and a_scores[-1] < a_scores[-2]:
    #     break
    
    conv_model_loss.append(conv_loss)
    torch.save(conv_model.state_dict(), "binary_model.pth")


    file = open("Convolutional_model_scores.txt", "w", encoding="utf8")
    file.write(f"losses: {conv_model_loss} \n")
    file.write(f"precisions: {p_scores} \n")
    file.write(f"recalls: {r_scores} \n")
    file.write(f"f1 scores: {f_scores} \n")
    file.write(f"accuracies: {a_scores} \n")
    file.close()

Epoch 1
-------------------------------


30it [05:47, 11.59s/it]


Convolutional Model Loss:  0.011058783996850252


10it [01:55, 11.50s/it]


Accuracy:  0.309375
prfa: 
0.12648221343873517 1.0 0.22456140350877193 tensor([0.3094])
Epoch 2
-------------------------------


...

Convolutional Model Loss:  0.007918744099636872


10it [01:53, 11.35s/it]


Accuracy:  0.43125
prfa: 
0.2865612648221344 0.9797297297297297 0.44342507645259943 tensor([0.4313])
Epoch 31
-------------------------------


27it [06:51, 15.25s/it]


KeyboardInterrupt: 

In [24]:
conv_model = Conv_NN()
conv_model.load_state_dict(torch.load("binary_model.pth", weights_only=True))

preds, labels = get_preds(test_loader, conv_model)
avg = (max(preds) + min(preds)) / 2
print(avg)
new_preds = get_clamped_preds(torch.flatten(preds), avg)
print(new_preds[0])
print(new_preds[1])
print(new_preds[2])

10it [02:05, 12.51s/it]

tensor([0.6546])
tensor(1)
tensor(1)
tensor(1)


In [39]:
from collections import Counter

print("Predictions \n--------------------")
print(new_preds[-100:])
# pred_one = 0
# pred_zero = 0
# for i in new_preds:
#     if i == 1:
#         pred_one += 1
#     else:
#         pred_zero += 1

# print("Predicted 1: ", pred_one)
# print("Predicted 0: ", pred_zero)
        
print("Actual \n--------------------")
print(labels[-100:].flatten())
# actual_one = 0
# actual_zero = 0
# for i in labels:
#     if i == 1:
#         actual_one += 1
#     else:
#         actual_zero += 1

# print("Actual 1: ", actual_one)
# print("Actual 0: ", actual_zero)

Predictions 
--------------------
tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 1])
Actual 
--------------------
tensor([1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])


In [ ]:
print(all_preds)

In [ ]:
print(all_labels[0].shape)

# Analysis
- Plot loss/accuracy (y) with number of epochs ran (x)
- Recall/Precision/F1 score 

In [ ]:
conv_model = Conv_NN()
conv_model.load_state_dict(torch.load("5th_conv_model.pth", weights_only=True))

test(test_loader, conv_model)

In [62]:
def graph_loss(x, y, metric):

    plt.plot(x, y)
    plt.xlabel("epoch #")
    plt.ylabel(f"{metric} score per epoch")
    plt.title(f"{metric} Scores")
    plt.savefig(f"{metric}_plot.png")
    plt.clf()

graph_loss(range(len(conv_model_loss)), conv_model_loss, "Cross Entropy Loss")
graph_loss(range(len(p_scores)), p_scores, "Precision")
graph_loss(range(len(r_scores)), r_scores, "Recall")
graph_loss(range(len(f_scores)), f_scores, "F1")
graph_loss(range(len(a_scores)), a_scores, "Accuracy")

<Figure size 640x480 with 0 Axes>

In [63]:
print("First vs Final Scores:")
print("Cross Entropy Loss:", conv_model_loss[0], conv_model_loss[-1])
print("Precision:", p_scores[0], p_scores[-1])
print("Recall:", r_scores[0], r_scores[-1])
print("F1:", f_scores[0], f_scores[-1])
print("Accuracy:", float(a_scores[0]), float(a_scores[-1]))

First vs Final Scores:
Cross Entropy Loss: 0.011058783996850252 0.007918744099636872
Precision: 0.12648221343873517 0.2865612648221344
Recall: 1.0 0.9797297297297297
F1: 0.22456140350877193 0.44342507645259943
Accuracy: 0.30937498807907104 0.4312500059604645


In [ ]:
# functions to show an image


def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(images[0])
# print labels
print(classes[labels[0]], images[0].shape)